In [47]:
from __future__ import print_function
import pandas as pd
import editdistance
from os import listdir
from os.path import isfile, join
import os
import numpy as np
import math

In [2]:
df = pd.read_excel('Excel Files with organizing data/Listen by book chapter.xlsx')

In [33]:
for col in df.columns: 
    print(col) 

Sequence of all items
Main Chapter heading
Chapter rank
Web chapter heading
Sequence of secondary chapters
Secondary chapter heading (too long to use)
Simplified secondary chapter heading
Web subchapter heading
Species name
Species file name
Explore numbers in this file
Numbered sounds in file
Book page
Photo #1
Caption for Photo #1
Photo #2
Caption for Photo #2
Photo #3
Caption for photo #3
sound 1
sound 2 
sound 3
sound 4
sound 5
sound 6
sound 7
sound 8
sound 9
sound 10
sound 11
sound 12
sound 13
sound 14
sound 15
sound 16
Unnamed: 35
Chapter rank.1
Chapter name
 Subchapter rank
 Subchapter heading


In [12]:
def get_actual_image_name(image_file, actual_images):
    if image_file in actual_images:
        return image_file
    else:
        distance = 10
        best = "ERROR"
        for im in actual_images:
            d = editdistance.eval(image_file, im)
            if d < distance:
                best = im
                distance = d 
#         print(best, image_file)
        return best

mypath = "../photos/"
actual_images = [f for f in listdir(mypath) if isfile(join(mypath, f))]
actual_images = map(lambda x: x.replace(".jpg",""), actual_images)


In [4]:
df_pages = pd.read_excel('Excel Files with organizing data/CURIOUS QR codes.xlsx')
for col in df_pages.columns: 
    print(col) 

Sequence of all items
URL for QR codes
SpeciesFile
Species name
Book page
Unnamed: 5
Corresponding to this URL


In [31]:
pages = {}

In [87]:
for index, row in df.iterrows():
    t = {}
    t['filename'] = row['Species file name']
    t['title'] = row['Species name']
    t['chapter_name'] = row['Web chapter heading'].replace(".",":")
    t['subchapter_name'] = row['Web subchapter heading'].replace('"',"&quot;").replace('--','&mdash;')
    t['from_page'] = row['Book page']
#     t['listen_by_number_heading'] = row['']
#     t['birds'] = []
    t['songs'] = row['''Numbered sounds in file''']
    t['pictures'] = []
    
    d = row
    if type(d["Photo #1"]) == unicode:
        photo_file = d["Photo #1"].replace(" ","_").replace("'","")    
        photo_file = get_actual_image_name(photo_file, actual_images)
        caption = d["Caption for Photo #1"]
        t['pictures'].append({'photo_file':photo_file, 'photo_caption':caption})
        
        if type(d["Photo #2"]) == unicode:
            photo_file = d["Photo #2"].replace(" ","_").replace("'","")    
            photo_file = get_actual_image_name(photo_file, actual_images)
            caption = d["Caption for Photo #2"]
            t['pictures'].append({'photo_file':photo_file, 'photo_caption':caption})
            
        if type(d["Photo #3"]) == unicode:
            photo_file = d["Photo #3"].replace(" ","_").replace("'","")    
            photo_file = get_actual_image_name(photo_file, actual_images)
            caption = d["Caption for photo #3"]
            t['pictures'].append({'photo_file':photo_file, 'photo_caption':caption})                    
    t['explores'] = str(row['Explore numbers in this file'])
    if t['explores'] == "nan":
        t['explores'] = ""
    pages[row['Sequence of all items']] = t
    


In [88]:
with open("yaml/pages.yaml", 'w') as f:
    f.write(yaml.safe_dump(pages, default_flow_style=False))

In [89]:
# df_species = pd.read_excel('Excel Files with organizing data/Listen by species.xlsx')
# for col in df_species.columns: 
#     print(col) 

In [90]:
birds = []

for index, row in df_species.iterrows():
    t = {}
    
    t['bird_name'] = row['Species']
    t['group'] = row['Group name']
    t['family'] = row['Family name']
    links = []
    
    for v in ['link {i}'.format(i=i) for i in range(1,6)]:
        x = '0'
        if type(row[v]) == unicode:
            x = row[v]
        elif not math.isnan(float(row[v])):   
            x = str(row[v])
        if x != '0':
            links.append(x)
    t['songs'] = links
    birds.append(t)
    


In [85]:
# df_species.head()

In [92]:
with  open("yaml/birds.yaml", 'w') as f:
    f.write(yaml.safe_dump(birds, default_flow_style=False))

In [99]:
df_explore = pd.read_excel(('Excel Files with organizing data/Explore.xlsx'))
for col in df_explore.columns: 
    print(col) 

Sequence
Group name
Explore file name
Explore name
Explore name--do not use
Species file
Move  this column left when files are all prepared; for now, previous column is a test column
Sequence.1
Web chapter heading
Web subchapter heading
page 
in book


In [100]:
explore = {}
for index, row in df_explore.iterrows():
    t = {}
    t['a_title'] = row['Explore name']
    t['chapter'] = row['Web chapter heading']
    t['subchapter'] = row['Web subchapter heading']
    t['x_from_page'] = row['''page 
in book''']
    explore[row['Sequence']] = t

In [101]:
with open("yaml/explore.yaml", 'w') as f:
    f.write(yaml.safe_dump(explore, default_flow_style=False))

In [103]:
# explore